In [ ]:
import sys
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'mapie',
    'causal',
    'wkmeans',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='halving',
        train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='random', search_type='wkmeans', search_subtype='wasserstein',
        n_models=10, n_jobs=1, debug=True,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 5000
else:
    n_trials = len(study.trials) + 5000

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

🔍 DEBUG: Parámetros sugeridos:
  cat_main_iterations: 742
  cat_main_depth: 9
  cat_main_learning_rate: 0.07939644054821023
  cat_main_l2_leaf_reg: 1.1268329184246422
  cat_main_early_stopping: 94
  cat_meta_iterations: 476
  cat_meta_depth: 6
  cat_meta_learning_rate: 0.03683238100402527
  cat_meta_l2_leaf_reg: 3.102553400001832
  cat_meta_early_stopping: 96
  max_main_periods: 3
  max_main_stats: 1
  method: first
  markup: 0.16319562840777277
  min_val: 2
  max_val: 15
  atr_period: 25
  periods_main: (40, 47, 83)
  stats_main: ('sharpe',)
  max_meta_periods: 1
  max_meta_stats: 1
  periods_meta: (7,)
  stats_meta: ('approxentropy',)
  n_clusters: 2
  window_size: 169
  step: 9
  max_iter: 425
  bandwidth: 1.8387787897225552
  n_proj: 16
🔍 DEBUG: base_df.shape = (42333, 5)
🔍 DEBUG: train_start = 2019-01-01 00:00:00, train_end = 2025-01-01 00:00:00
🔍 DEBUG: test_start = 2022-01-01 00:00:00, test_end = 2023-06-01 00:00:00
🔍 DEBUG: pad = 90
🔍 DEBUG: bar_delta = 0 days 01:00:00
🔍 DEBUG: